# Web Science Project Week 3+4 on Network Dynamics

Emiel Steegh   - s1846388  
Freek Nijweide - s1857746

## Project Description

For this week's assignments, we will examine [...]

We use several packages this week,please make sure these are installed when trying to run our code:
- networkx
- numpy
- plotly
- pygraphviz

We decided to use plotly instead of matplotlib, due to [...]

its vastly superior 3D plotting capabilities, and its interactivity. We hoped that this would increase the legibility of our results, and we think that it has done so. We came to various conclusions for assignment 3 that we could not have reached without the assistance of interactive 3D plots.

# TODO comment code

# TODO remove comments zoals "wacky imports die emiel denkt.."

In [62]:
# The following includes are needed to work with graphs and display solutions.
from __future__ import division
import networkx as nx

from IPython.display import SVG
from IPython.display import HTML
from IPython.display import display

import StringIO

from networkx.drawing.nx_pydot import read_dot
from networkx.drawing.nx_pydot import from_pydot
from networkx.drawing.nx_agraph import to_agraph
import pydot

import numpy as np
import random

#wacky imports die emiel denkt nodig te hebben
import glob #om file paths en zo te fixen
import pprint #debugging purposes
import pandas as pd #dataframes en zo voor plotly
import os #path handling
import json #het lezen van die files
#coolio
import dateutil.parser

try:
    import plotly.graph_objects as go
    import plotly.colors
except:
    print("\n"+
          "┌---------------- (!) Warning (!) ----------------┐")
    print("|                                                 |")
    print("| \x1b[31m  It looks like Plotly could not be imported,\x1b[0m   |")
    print("|   please make sure it is properly installed.    |")
    print("|                                                 |")
    print("└-------------------------------------------------┘")
else:
    print("\x1b[32mimports successful")

imports successful


In [37]:
#credits for the following function go to Benjamin Toueg
#https://stackoverflow.com/questions/16888409/suppress-unicode-prefix-on-strings-when-using-pprint
def my_safe_repr(object, context, maxlevels, level):
    typ = pprint._type(object)
    if typ is unicode:
        object = str(object)
    return pprint._safe_repr(object, context, maxlevels, level)

pp = pprint.PrettyPrinter()
pp.format = my_safe_repr

def ANSI_col_code(color):
    code = 31
    if type(color) not in [str, float]:
        code = 31
    elif color.lower() in ['r', "red"]:
        code = 31
    elif color.lower() in ['g', "green"]:
        code = 32
    elif color.lower() in ['b', "blue"]:
        code = 94
    elif color.lower() in ['y', "yellow"]:
        code = 33
    elif color.lower() in ['m', "magenta", 'p', "purple"]:
        code = 35
    return code

def cprint(string, color = 'r'):
    code = ANSI_col_code(color)
    print "\x1b[{}m{}\x1b[0m".format(code, string)

def cstring(string, color = 'r'):
    code = ANSI_col_code(color)
    return "\x1b[{}m{}\x1b[0m".format(code, string)

cprint("Red", 'r')
cprint("Green", 'g')
cprint("Blue", 'b')
cprint("Yellow", 'y')
cprint("Purple", 'm')
print("back to normal")

plotly_colors = [ # copied from https://community.plot.ly/t/plotly-colours-list/11730/2
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]


Red
Green
Blue
Yellow
Purple
back to normal


## Data read, parse & store

In [3]:
def filetype_paths (data_folder, filetype='json'):
    '''
    returns a list of paths of the *.json files in a dir (data_folder)
    '''
    target = os.path.join(data_folder, '*.{}'.format(filetype))
    paths = glob.glob(target)
    return paths

def path_to_timestamp(path):
    '''
    turns yyyymmdd into yyyy-mm-dd
    '''
    dateraw = ((os.path.split(path)[1])[:8])
    date = "{}-{}-{}".format(dateraw[:4], dateraw[4:6], dateraw[6:8])
    return date


def data_from_youtube(folder):
    #grab the list of files to parse
    ls_locations = filetype_paths(folder, 'json')
    
    #data frame that will conatain all relevant data
    df = pd.DataFrame(columns=['timestamp', 'song', 'youtube_id',  'release_date', 'views', 'votes', 'likes', 'dislikes'])
    
    #dict conatining the different collumns for appending data
    new_row = {'timestamp' : "_", 'release_date' : "_", 'song' : "_", 'youtube_id' : "_", 'views' : 0, 'votes' : 0, 'likes' : 0, 'dislikes' : 0}
    
    #loops through all files in the data folder
    for loc in ls_locations:
        
        #grabs datagather timestamp from file title
        new_row['timestamp'] = path_to_timestamp(loc)
        file_raw_data =json.load(open(loc))
        
        #take all the other data from the relevant locations in the file
        for entry in file_raw_data:
            new_row['song'] = (entry["snippet"])["title"]
            new_row['youtube_id'] = entry["id"]
            new_row['release_date'] = ((entry["snippet"])["publishedAt"])[:10]
            stats = entry["statistics"]
            new_row['views'] = int(stats["viewCount"])
            new_row['likes'] = int(stats["likeCount"])
            new_row['dislikes'] = int(stats["dislikeCount"])
            new_row['votes'] = new_row['likes']+new_row['dislikes']
            
            df = df.append(new_row, ignore_index=True)
            
    return df

def data_from_spotify(folder):
    
    ls_locations = filetype_paths(folder, 'json')
    
    df = pd.DataFrame(columns = ['timestamp', 'song', 'spotify_id', 'release_date', 'popularity','duration_ms'])
    
    new_row = {'timestamp' : "_", 'release_date' : "_", 'song' : "_", 'spotify_id' : "_", 'popularity' : 0, 'duration_ms':0}
    
    for loc in ls_locations:
        new_row['timestamp'] = path_to_timestamp(loc)
        file_raw_data = json.load(open(loc))
        
        items = (file_raw_data["tracks"])["items"] # the list of song-datas in list format
        for item in items:
            track = item["track"]
            artists = ""
            for artist in (track["artists"]):
                add_artist = (artist["name"]).encode('utf-8')#unicode(artist["name"], 'ascii')
                if artists == "":
                    artists = add_artist
                else:
                    artists += ", {}".format(add_artist)
                    
            new_row['song'] = "{} - {}".format(artists, (track["name"]).encode('utf-8'))
            new_row['spotify_id'] = track["id"]
            new_row['popularity'] = track["popularity"]
            new_row['duration_ms'] = track['duration_ms']
            new_row['release_date'] = (item["added_at"])[:10]            
            
            df = df.append(new_row, ignore_index=True)
    
    return df
            
    
def create_and_store_data():
    '''
    Wrapper of the read and parse function, stores the desired data from the 4 folders in 4 seperate .csv files
    The reading all the data files and wrangling the data into a useful format takes a bunch of your time, we dont want that.
    This definition does all that once and stores the created dataframes as csv files in a new folder.
    This way the process only has to happen once and on all consecutive runs of the notebook, we just have to read 4 files that contain the data we want and in the way we want it
    
    please only run it through def check_or_create_data
    '''
    location = os.path.join('data','spotify_top100') #spotify top 100
    df_spotify_top100 = data_from_spotify(location)
    df_spotify_top100.to_csv(os.path.join('csv_data', 'wrangled_spotify_top100.csv'), index=False, sep='\t', encoding='utf-16')
    cprint("set (1/4) done: spotify",'y')
    #pp.pprint(df_spotify_top100)

    location = os.path.join('data','youtube_top100') #youtube top 100
    df_youtube_top100 = data_from_youtube(location)
    df_youtube_top100.to_csv(os.path.join('csv_data', 'wrangled_youtube_top100.csv'), index=False, sep='\t', encoding='utf-16')
    cprint("set (2/4) done: youtube",'y')
    #pp.pprint(df_youtube_top100)

    location = os.path.join('data','radio3fm_megahit') #radio 3fm popular track of the day
    df_3fm_megahit = data_from_youtube(location)
    df_3fm_megahit.to_csv(os.path.join('csv_data', 'wrangled_3fm_megahit.csv'), index=False, sep='\t', encoding='utf-16' )    
    cprint("set (3/4) done: radio 3fm",'y')
    #pp.pprint(df_3fm_megahit)

    location = os.path.join('data','radio538_alarmschijf') #radio 538 tracks that may become popular??
    df_538_alarmschijf  = data_from_youtube(location)
    df_538_alarmschijf.to_csv(os.path.join('csv_data', 'wrangled_538_alarmschijf.csv'), index=False, sep='\t', encoding='utf-16')    
    cprint("set (4/4) done: radio 538",'y')
    #pp.pprint(df_538_alarmschijf)
    
data_megahit      = 'global'
data_alarmschijf  = 'global'
data_youtube100   = 'global'
data_spotify100   = 'global'


def check_or_create_data():
    '''
    Checks wether the required data files are present, if they are not runs the func to create them
    '''
    
    if not os.path.exists('csv_data'):
        os.mkdir('csv_data')
    
    csv_goal = [os.path.join('csv_data', 'wrangled_3fm_megahit.csv'),
           os.path.join('csv_data', 'wrangled_538_alarmschijf.csv'),
           os.path.join('csv_data', 'wrangled_youtube_top100.csv'),
           os.path.join('csv_data', 'wrangled_spotify_top100.csv')]
    csv_present = filetype_paths('csv_data', 'csv')

    if sorted(csv_present) == sorted(csv_goal):
        cprint("all data files seem present!",'g')
    else:
        cprint("data is missing and will be created, this may take a while", 'r')
        create_and_store_data()
        cprint("all done!", 'g')
    global data_megahit, data_alarmschijf, data_youtube100, data_spotify100
    data_megahit = pd.read_csv(os.path.join('csv_data', 'wrangled_3fm_megahit.csv'), delimiter = '\t', encoding='utf-16')
    data_alarmschijf = pd.read_csv(os.path.join('csv_data', 'wrangled_538_alarmschijf.csv'), delimiter = '\t', encoding='utf-16')
    data_youtube100 = pd.read_csv(os.path.join('csv_data', 'wrangled_youtube_top100.csv'), delimiter = '\t', encoding='utf-16')
    data_spotify100 = pd.read_csv(os.path.join('csv_data', 'wrangled_spotify_top100.csv'), delimiter = '\t', encoding='utf-16')

    

In [4]:

check_or_create_data()

testprint = False
if testprint:
    print "\n\n"
    cprint("--- --- Radio3fm Megahit:",'p')
    print data_megahit
    cprint("--- --- Radio538 Alarmschijf:",'p')
    print data_alarmschijf
    cprint("--- --- Spotify Top100 Dataset:",'p')
    print data_spotify100
    cprint("--- --- Youtube Top100 Dataset:",'p')
    print data_youtube100
    

# location = os.path.join('data','spotify_top100') #spotify top 100
# df_spotify_top100 = data_from_spotify(location)
# df_spotify_top100.to_csv(os.path.join('csv_data', 'wrangled_spotify_top100.csv'), index=False, sep='\t', encoding='utf-16')


all data files seem present!


## Assignment 1 - Cascading effects

include:
- plot
- code explanation

Read the material in Chapter 16 on cascading effects. Plot the difference between the number of likes and dislikes for several songs.

Do you think we observe cascading effects? Is there a difference between songs that are already popular (in the top-100) and those that are not (megahit or alarmschijf)?. The model in Chapter 16 in its pure form cannot be applied to music preferences. Why not? Can you modify the model accordingly? Can you quantify cascading effects in this setting?

There is no one right answer right to this question, and the answer may depend on a particular song. Make your conclusions based on your data and present clear arguments.

# TODO explain drake jumpman. Nummer met laag aantal views, duidelijke cascading.

In [107]:
#TODO plot likes vs dislikes for several songs categorizeb by popularity
#TODO why cant the ch 16 model bot be applied to songs (int vs bool)
#TODO change model to work for songs
#TODO quantify cascading effect
#TODO comment code
#TODO pretty graphs

def find_songs(dataframe,mode='relative_likes'):
    dt_songs = {}
    
    for index, row in sorted(dataframe.iterrows(), key=lambda k: k[1]['timestamp']):
        y_id = row['song']
        if mode=='relative_likes':
            data = int(row['likes'])-int(row['dislikes'])
        elif mode=='like_ratio':
            data = int(row['likes'])/int(row['votes'])
        elif mode=='expectation':
            data = int(row['views'])
        else:
            data = int(row[mode])
            
        
        if mode=='expectation':
            data2 = int(row['likes'])
        else:
            data2 = row['timestamp']
        
        if y_id not in dt_songs.keys():
            dt_songs[y_id] = [[data],[data2]]
        else:
            dt_songs[y_id][0].append(data)
            dt_songs[y_id][1].append(data2)
                        
    return(dt_songs)
     


def draw_linechart(figure,dt_data,plot_trendline=False,**kwargs):
    colors=plotly.colors.DEFAULT_PLOTLY_COLORS
    i=0    
    if plot_trendline:
        all_x = [data[1] for song, data in dt_data.items()]
        flattened_x = [item for sublist in all_x for item in sublist]
    for song, data in dt_data.items():
        color = colors[i]
        kwargs['y'] = data[0]
        kwargs['x'] = data[1]
        kwargs['name']=song
        mode='markers+lines'
        line={'width':0.5,'color':color}
        marker={'size':3,'color':color}
        try:
            mode = kwargs['mode']
        except:
            pass
        try:
            line = kwargs['line']
        except:
            pass
        try:
            marker = kwargs['marker']
        except:
            pass
        figure.add_trace(go.Scatter(kwargs,mode=mode,line=line,marker=marker))
        
        
        if plot_trendline:
            trendline = np.poly1d(np.polyfit(kwargs['x'],kwargs['y'],1))
            figure.add_trace(go.Scatter(x=flattened_x,y=trendline(flattened_x),mode='lines',name='Trendline for' + song,line={'width':0.3,'color':color,'dash':'dot'}))

        i = (i + 1) % (len(colors)-1 ) 
        
    
def get_n_songs(data,n=None):
    if n:
        return data[data.song.isin(data.song.sample(n=n))]
    else:
        return data
    
data_set = pd.concat([get_n_songs(data_youtube100,n=10),data_youtube100[data_youtube100.song.str.contains('Drake - Jumpman')]] )

data1 = find_songs(data_set,mode='relative_likes')
figure = go.Figure()
draw_linechart(figure,data1,mode='markers+lines',line={'width':0.5},marker={'size':3})
figure.update_layout(legend=dict(x=0, y=-2))
figure.show()

data2 = find_songs(data_set,mode='like_ratio')
figure2 = go.Figure()
draw_linechart(figure2,data2)
figure2.update_layout(legend=dict(x=0, y=-2))
figure2.show()

## Assignment 2 - Network effects

include  at least:
- plot
- code explanation

*Study the material in Chapter 17. Our goal now is to investigate whether we observe network effects in music preferences. How will you choose the data for this purpose? Which songs will suit most?*

The songs that will suit most are ones that we can track their rise and fall of popularity of from the moment they were published. The most suitable candidates are those that have been a 3FM megahit or on the 538 alarmschijf, as they were quite unknown when being published on there. To compare, we will choose some songs from the Youtube and Spotify top 100 list. For these songs, the popularity / viewcount is the most interesting data to plot over time.

*For several songs of your choice plot the actual number of views against time. Assume that without network effects we can expect that users visit a website with a certain frequency and view the song if it matches their taste. Hence, the expected number of views grows linearly in time. Compare your plot to Figure 17.4. Do you think you observe network effects?*

# TODO plot titles enzo

We plot the viewcount / popularity of the songs mentioned above.

In [108]:
some_youtube_songs = data_youtube100[data_youtube100.song.isin(data_youtube100.song.head(n=5))]  
some_spotify_songs = data_spotify100[data_spotify100.song.isin(data_spotify100.song.head(n=5))]  

unknown_songs_figure = go.Figure()
youtube_songs_figure = go.Figure()

alarmschijf_data = find_songs(data_alarmschijf,mode='views')
draw_linechart(unknown_songs_figure,alarmschijf_data,mode='lines',line={'dash':'dot'},legendgroup='alarmschijf')

megahit_data = find_songs(data_megahit,mode='views')
draw_linechart(unknown_songs_figure,megahit_data,mode='lines',legendgroup='megahit')

some_youtube_songs_data = find_songs(some_youtube_songs,mode='views')
draw_linechart(youtube_songs_figure,some_youtube_songs_data,mode='lines',line={'dash':'dot'},legendgroup='some_youtube_songs')

some_spotify_songs_data = find_songs(some_spotify_songs,mode='popularity')
draw_linechart(youtube_songs_figure,some_spotify_songs_data,plot_trendline=False,yaxis='y2',legendgroup='some_spotify_songs')



unknown_songs_figure.show()
youtube_songs_figure.update_layout(    
    yaxis=dict(title="Youtube views")
    
    ,yaxis2=dict(
        title="Spotify popularity",
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    ),
legend=dict(x=0, y=-2)
)
youtube_songs_figure.show()
# ex2_plot.add_trace(go.Scatter(x=[dateutil.parser.parse(x) for x in unknown_hits.timestamp],y=[int(viewcount) for viewcount in unknown_hits.views]))


### Plot explanation

The 3FM Megahit and 538 Alarmschijf songs all seem to have quite a linear relationship between time and popularity, while the more popular songs from the top 100 have a relationship between time and popularity which is very similar to figure 17.4 (a fast rise at first, then a more slow stabilization towards the equilbrium).

Thus, we can see quite clearly that network effects are not visible for impopular songs (users simply come across it by visiting a platform and clicking songs that match their preferences), while they are visible for more popular songs (people listen to them because of their popularity).

### Assignment 2 (continued)

*How will you interpret the network benefit function f, the intrinsic interest function r, and the price p*? Try to specify the model that best fits the data.*

For plots like for functions f and r, normally the share of consumers that buys something is plotted on the x axis, and hte y axis represents the "price" of the good. In this case, the share of consumers can be represented by the amount of views (amount of people that took the time to click on the video). Then, what do we plot on the y-axis? 

# TODO leg alle plots hieronder uit
# TODO zeg: dislike was het tijd niet waard, like was het tijd wel waard. Dus bij hoge dislike ratio was de prijs te hoog

In [9]:
figure = go.Figure()
merged_data = pd.concat([data_youtube100,data_alarmschijf,data_megahit])
# merged_data = data_megahit
data = merged_data[merged_data.timestamp == u'2016-11-28']
# data = data_youtube100

dislikes_over_votes = (data.dislikes/data.votes)
likes_over_votes = data.likes/data.votes
likes_over_dislikes = data.likes/data.dislikes

# x axis: population share
# y axis: estimate

figure.add_trace(go.Scatter(x=data.views,y=data.likes/data.views,mode='markers',name='Likes per view',marker={'color':'royalblue'}))
figure.add_trace(go.Scatter(x=data.views,y=likes_over_votes,mode='markers',name='Like ratio',yaxis='y2',marker={'color':'tomato'}))
# figure.add_trace(go.Scatter(x=data.views,y=data.duration_ms,mode='markers',name='duration',yaxis='y3'))

trendline1 = np.poly1d(np.polyfit(data.views,data.likes/data.views,1))
trendline2 = np.poly1d(np.polyfit(data.views,likes_over_votes,1))

figure.add_trace(go.Scatter(x=data.views,y=trendline1(data.views),mode='lines',name='Trendline 1',line={'color':'royalblue'}))
figure.add_trace(go.Scatter(x=data.views,y=trendline2(data.views),mode='lines',name='Trendline 2',line={'color':'tomato'},yaxis='y2'))

# data_youtube100



# x axis: expectation
# y axis: realization
trendline1 = np.poly1d(np.polyfit(likes_over_votes,data.views,1))
trendline2 = np.poly1d(np.polyfit(likes_over_votes,np.array(data.likes/data.views),1))

figure2 = go.Figure()
figure2.add_trace(go.Scatter(x=likes_over_votes,y=data.views,mode='markers',name='Views',marker={'color':'royalblue'}))
figure2.add_trace(go.Scatter(x=likes_over_votes,y=trendline1(likes_over_votes),mode='lines',name='Trendline 1',line={'color':'royalblue'}))
figure2.add_trace(go.Scatter(x=likes_over_votes,y=data.likes/data.views,mode='markers',name='Likes per view',yaxis='y2',marker={'color':'tomato'}))
figure2.add_trace(go.Scatter(x=likes_over_votes,y=trendline2(likes_over_votes),mode='lines',name='Trendline 2',yaxis='y2',line={'color':'tomato'}))


# figure2.add_trace(go.Scatter(x=likes_over_votes,y=data.views,mode='markers',name='Likes per view'))

figure.update_layout(    
    yaxis=dict(title="Likes per view")
    
    ,yaxis2=dict(
        title="Like ratio",
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    ),
    
# legend=dict(x=0, y=-2)
)

figure.show()

figure2.update_layout(    
    yaxis=dict(title="Views")
    
    ,yaxis2=dict(
        title="Likes per view",
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    ),
# legend=dict(x=0, y=-2)
)

figure2.show()
# print data_youtube100.likes / data_youtube100.votes

# TODO expectation vs reality plot: nummers over tijd. Views op y as, iets met likes op x as. Explain


# TODO arrows

# TODO de trendline: slope van trendline is hoeveel mensen willing zijn te kopen?



In [109]:
# figure = go.Figure()

songs=get_n_songs(data_youtube100,n=3)
# figure.add_trace(go.Scatter(x=data.likes,y=data.views,mode='markers'))


data1 = find_songs(songs,mode='expectation')
figure = go.Figure()
draw_linechart(figure,data1,True)
# trendline1 = np.poly1d(np.polyfit(songs.likes,songs.views,1))
# figure.add_trace(go.Scatter(x=songs.likes,y=trendline1(songs.likes),line={'color':'black','dash':'dash','width':3}))
# figure.update_layout(legend=dict(x=0, y=-2))
figure.update_layout(showlegend=False)
figure.show()



# TODO dit had positief moeten zijn????

songs2=get_n_songs(data_youtube100,n=None)
# figure.add_trace(go.Scatter(x=data.likes,y=data.views,mode='markers'))


data2 = find_songs(data_youtube100,mode='expectation')
figure2 = go.Figure()
draw_linechart(figure2,data2)
trendline2 = np.poly1d(np.polyfit(songs.likes,songs.views,1))
figure2.add_trace(go.Scatter(x=songs.likes,y=trendline2(songs.likes),line={'color':'black','dash':'dash','width':3}))
# figure.update_layout(legend=dict(x=0, y=-2))
figure2.update_layout(showlegend=False)
figure2.show()

In [34]:
# Plotting duration against sum of all instances of song's popularity (discrete version of integral of popularity over time)

figure = go.Figure()
data = get_n_songs(data_spotify100,n=None)
    
max_popularities=( [ (data[data.spotify_id == spotify_id].duration_ms.iloc[0] , sum(list(data[data.spotify_id == spotify_id].popularity)) ) for spotify_id in data.spotify_id.unique()])
ls_durations, ls_max_popularities = map(list, zip(*max_popularities))
data = data[data.timestamp == u'2015-12-15']


figure.add_trace(go.Scatter(x=ls_max_popularities,y=ls_durations,mode='markers',name='Song popularity',marker={'color':'royalblue'}))

trendline1 = np.poly1d(np.polyfit(ls_max_popularities,ls_durations,1))
figure.add_trace(go.Scatter(x=ls_max_popularities,y=trendline1(ls_max_popularities),mode='lines',name='Trendline 1',line={'color':'royalblue'}))


figure.show()



# TODO explain graphs above

## Assignment 3 - Popularity effects

include at least:
- plot
- code explanation

Study the material in Chapter 18. We investigate whether rich-get-richer phenomenon explains the dynamics of the number of views.

*Plot the distribution of the number of views among the songs on several different days. Do you observe power laws?*

For this, we will add show 3 plots, with youtube data (because the question mentions *views* and not listens or popularity), from randomly selected dates.

We start with a linear plot of the data, with the trendline from the lin-log plot.

Next we show a log-lin plot of the data, with a trendline calculated. Straight lines in log-lin plots show exponential relations.
We also added the trendline from the log-log plot (dashed) to illustrate the difference.
(We do not show the trendline created from the log-log plot in the linear plot, because of the enormous difference in scale this causes)

Finally, we show a log-log plot of the data, with a trendline calculated for it (dashed line). Straight lines in log-log plots show power law relationships.
We also added the trendline from the lin-log plot to illustrate the difference (non-dashed).

In [ ]:
# Plotting data on a certain day

dates = [u'2015-11-09', u'2016-04-13',u'2016-09-16']
colors=['tomato','royalblue','gold']

day1_fig = go.Figure()
day1_fig_log = go.Figure()
day1_fig_loglog = go.Figure()
for date in dates:
    data = sorted(data_youtube100[data_youtube100.timestamp == date].views,reverse=True)
    color = colors[dates.index(date)]
    x = np.array(range(len(data))) + 1
    polynomial_coefficients_log, residuals_log = np.polyfit(x,np.log(data), 1,full=True)[:2] # Returns coefficients of a polynomial of degree 1 (just a linear relation) with least square fit to data
    polynomial_coefficients_loglog, residuals_loglog = np.polyfit(np.log(x),np.log(data), 1,full=True)[:2] # Returns coefficients of a polynomial of degree 1 (just a linear relation) with least square fit to data
    
    print 'Sum of the squares of the fit errors for ' + date + ' lin-log trendline: ' + str(round(residuals_log[0],2))
    print 'Sum of the squares of the fit errors for ' + date + ' log-log trendline: ' + str(round(residuals_loglog[0],2))
    
    polynomial_name_log = 'Exponential fit: e^(' + str(round(polynomial_coefficients_log[0],2)) + 'x) + e^' + str(round(polynomial_coefficients_log[1],2))
    polynomial_name_loglog = 'Power law fit: e^' + str(round(polynomial_coefficients_loglog[1],2)) + ' * x^' + str(round(polynomial_coefficients_loglog[0],2))

    
    polynomial_log = np.poly1d(polynomial_coefficients_log) # Generate a polynomial from these coefficients
    polynomial_loglog = np.poly1d(polynomial_coefficients_loglog) # Generate a polynomial from these coefficients

    date_name = 'Data from ' + date
    
    # Plot trend line

    day1_fig.add_trace(go.Scatter(x=x,y=data,mode='markers',marker={'color': color,'size':3},name=date_name,legendgroup=date))
    day1_fig.add_trace(go.Scatter(x=x,y=np.exp(polynomial_log(x)),marker={'color': color},line={'width' : 0.5},name=polynomial_name_log,legendgroup=date))

    day1_fig_log.add_trace(go.Scatter(x=x,y=data,mode='markers',marker={'color': color,'size': 3},name=date_name,legendgroup=date))
    day1_fig_log.add_trace(go.Scatter(x=x,y=np.exp(polynomial_log(x)),marker={'color': color},line={'width' : 0.5},name=polynomial_name_log,legendgroup=date))
    day1_fig_log.add_trace(go.Scatter(x=x,y=np.exp(polynomial_loglog(np.log(x))),line={'dash': 'dashdot','color': color,'width':1.5},name=polynomial_name_loglog,legendgroup=date))

        
    day1_fig_loglog.add_trace(go.Scatter(x=x,y=data,mode='markers',marker={'color': color,'size':3},name=date_name,legendgroup=date))
    day1_fig_loglog.add_trace(go.Scatter(x=x,y=np.exp(polynomial_log(x)),line={'color': color,'width' : 0.5},name=polynomial_name_log,legendgroup=date))
    day1_fig_loglog.add_trace(go.Scatter(x=x,y=np.exp(polynomial_loglog(np.log(x))),line={'dash': 'dashdot','color': color,'width':1.5},name=polynomial_name_loglog,legendgroup=date))
    
    
#     day1_fig.add_trace(go.Scatter(x=x,y=np.exp(polynomial_loglog(np.log(x)))))
#     day1_fig.add_trace(go.Scatter(x=x,y=np.exp(polynomial(np.log(x)))))

day1_fig.update_layout(title='Linear plot for song view count vs ranking',
                    xaxis_title='Sorted ranking by viewcount of song on that day',
                    yaxis_title='View count of song')
day1_fig.show()
day1_fig_log.update_layout(title='Log-lin plot for song view count vs ranking',
                    xaxis_title='Sorted ranking by viewcount of song on that day',
                    yaxis_title='View count of song',
                    yaxis_type='log')
day1_fig_loglog.update_layout(title='Log-log plot for song view count vs ranking',
                    xaxis_title='Sorted ranking by viewcount of song on that day',
                    yaxis_title='View count of song',
                    xaxis_type='log',
                    yaxis_type='log')
day1_fig_log.show()
day1_fig_loglog.show()
# day1_fig2 = go.Figure()
# day1_fig2.add_trace(go.Scatter(x=x,y=day1_data,mode='markers',color='red'))
# # day1_fig2.add_trace(go.Scatter(x=x,y=np.exp(polynomial(np.log(x)))))
# day1_fig2.update_layout(yaxis_type="log")
# day1_fig2.show()


### Explanation of plots:

The fit of the lin-log plot seems to be better than the fit on the log-log plot. This, combined with the large amount of outliers for the trendlines on the log-log plot, seems to suggest that the distribution might be an exponential one (of the form $b + a^{cx}$ ) instead of a power law (of the form $b \cdot x^{-c}$ ). The mean squared error (printed above the plots) for the lin-log trendline is far lower than the error for the log-log trendline.

While we believe that an exponential relationship is more likely than a power law, one can also see that the trendline for a power-law is is an acceptable fit for the data, with c ranging between 1.5 and 2 (quite a weak power law).

### Exercise 3 (continued)

*Assume that the number of views in the next day is proportional to the total number of views up to the day before. Argue that in this case the number of views will grow exponentially in time.*

This is quite easy to prove. Let's say the amount of views of song $i$, $v_i$, is multiplied by some constant $x_i$ for that song, each day.

After $t$ days, the amount of views is:

$v_i (t) = v_{i_{initial}} \cdot {x_i}^t $

There is an obvious exponential correlation between time and the number of views.

*Look again at the plots for the number of views over time that you produced in assignment 2. Do we observe exponential growth on data? Maybe we observe exponential growth at least some periods of time? Do you think we observe the rich get richer phenomenon?*

# TODO assignment 2, daarna dit

*Compare the ranking of songs in the Spotify data set (based on their position in the top-100) with the ranking of songs in the YouTube data set (based on their number of views) over time. Are the outcomes in line? Why (not)?*

# TODO previous ding, daarna dit

## Assignment 4 - Information diffusion

Read Chapter 19. Do you think the model of information diffusion applies for music preferences? How can you observe this on the data? Is it related to other phenomena discussed above? Again, there is no one right answer to these questions, try to formulate your own ideas.

## Assignment 5 - Create your own data

include at least:
- plot
- code explanation

Use the YouTube API to collect your own data set to investigate distribution of popularity and the long tail phenomenon. Start with obtaining the number of views of a song of your choice. Make a random selection from the recommendations that come with this song, and obtain the number of views of this recommended song. Repeat this sequence at least 100 times. Does this data set illustrate the strong variation in the market share of different songs (stronger than would be expected on basis of a normal distribution)? Can you observe the long tail in the distribution of popularity? Visualize the distribution with a graph similar to Figure 18.4.

A document with instructions on how to use the YouTube API is available on Blackboard. The YouTube API will return results in JSON format, so your data set will be a JSON file.

## Assignment 6 - Conclusions

Make conclusions: 1) which effects and models explain best the data on music preferences, 2) which data we need to collect if we want to investigate cascading, network and rich-get-richer effects in music preferences?